# Using EcoFOCIpy to process raw field data

## DY2103 (Spring Mooring Cruise - Oscar Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2021/CTDcasts/dy2103/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [16]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY2103' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY2103.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTDplusrinko.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

In [17]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd001.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sbeox1ml/l,sbeox1ps,sbox1mm/kg,sal00,sal11,sigma-t00,sigma-t11,fleco-afl,t090c,t190c,turbwetntu0,prdm,scan
count,8.000000,8.000000,8.000000,8.000000,8.00000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,7.128713,97.633375,310.531375,6.992538,95.76800,304.598750,31.864712,31.868725,25.237938,25.241163,1.028050,4.550150,4.549812,0.389387,81.227375,6015.875000
std,0.006468,0.088190,0.280635,0.006352,0.08764,0.275937,0.006441,0.006724,0.005182,0.005408,0.367768,0.000812,0.000853,0.115533,0.218293,114.081972
min,7.120000,97.517000,310.151000,6.988000,95.70500,304.402000,31.852100,31.854100,25.227800,25.229400,0.545500,4.548900,4.548600,0.196900,80.894000,5858.000000
25%,7.125275,97.587000,310.383000,6.988375,95.70875,304.418000,31.862000,31.868825,25.235775,25.241225,0.756250,4.549850,4.549450,0.330125,81.164000,5936.750000
50%,7.127800,97.619000,310.492500,6.989050,95.72200,304.446500,31.866200,31.870250,25.239100,25.242400,1.041350,4.550050,4.549650,0.382550,81.270500,6003.500000
75%,7.131525,97.671250,310.653500,6.994775,95.80100,304.694750,31.867675,31.870875,25.240300,25.242875,1.268225,4.550525,4.550025,0.482825,81.332250,6095.250000
max,7.138700,97.771000,310.965000,7.005200,95.94200,305.150000,31.873700,31.877600,25.245200,25.248300,1.512100,4.551500,4.551500,0.545300,81.479000,6190.000000


## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [18]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY2103',
 'CruiseID_Historic': None,
 'CruiseID_Alternates': None,
 'Project_Leg': '',
 'Vessel': 'R/V Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2021, 5, 1),
 'EndDate': datetime.date(2021, 5, 20),
 'Project': 'FOCI',
 'ChiefScientist': 'Geoff Lebon',
 'StartPort': 'Kodiak, AK',
 'EndPort': 'Kodiak, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'FOCI Spring Mooring Cruise',
 'CruiseYear': 2021,
 'ctdlogs_pdf_name': 'DY2103_CTDCastLogs.pdf'}

In [19]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 24484,
 'Vessel': 'R/V Oscar Dyson',
 'CruiseID': 'DY2103',
 'Project_Leg': '',
 'UniqueCruiseID': 'DY2103',
 'Project': 'Spring Mooring Cruise',
 'StationNo_altname': 'S1H4',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 54,
 'LatitudeMin': 31.24,
 'LongitudeDeg': 165,
 'LongitudeMin': 10.42,
 'GMTDay': 3,
 'GMTMonth': 'MAY',
 'GMTYear': 2021,
 'GMTTime': 61980,
 'DryBulb': 2.2,
 'RelativeHumidity': 60,
 'WetBulb': -99.9,
 'Pressure': 1011,
 'SeaState': '-99',
 'Visibility': '-99',
 'WindDir': 262,
 'WindSpd': 19.0,
 'CloudAmt': '-99',
 'CloudType': '-99',
 'Weather': '-99',
 'SurfaceTemp': -99.9,
 'BottomDepth': 90,
 'StationNameID': '',
 'MaxDepth': 81,
 'InstrumentSerialNos': 'Press SN = 0291, Pri Temp SN = SBE3-4379 , Sec Temp SN = SBE3-2376, Pri Cond SN = SBE4-2985 , Sec Cond Sn = SBE4-3127, PAR Sn = 70497, Fluor Sn = FLNTU-2057, pri O2 Sn = SBE43-1961 , sec O2 Sn =  SBE43-1876 , ch6/7 = rinko optode',
 'Notes': 'Snow squalls in vicinity, large swells. Test and training 

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [20]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time',
  'time_origin': '1900-01-01 00:00:00',
  'units': 'days since 1900-01-01T00:00:00Z'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'ge

In [45]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['ctd001.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,turbidity,Pressure [dbar],scan,datetime
bottle,,,,,,,,,,,,,,,,,
5.0,7.12,97.517,310.151,6.9934,95.783,304.635,31.8737,31.8776,25.2452,25.2483,0.5455,4.5489,4.5486,0.5453,81.453,6022.0,2021-05-03 09:10:58


In [22]:
cruise_data['ctd001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'salinity_ch1', 'salinity_ch2',
       'sigma_t_ch1', 'sigma_t_ch2', 'chlor_fluorescence', 'temperature_ch1',
       'temperature_ch2', 'turbidity', 'Pressure [dbar]', 'scan', 'datetime'],
      dtype='object')

## Add institutional meta-information


In [23]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [63]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [67]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/17)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 7.13 ... ...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 97.65 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 310.6 ......
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 6.988 ......
    oxy_percentsat_ch2  (latitude, longitude, time, bottle) float64 95.7 ... ...
    oxy_conc_ch2        (latitude, longitude, time, bottle) float64 304.4 ......
    ...                  ...
    temperature_ch1     (latitude, longitude, time, bottle) float64 4.551 ......
    temperature_ch2     (latitude, longitude, time, bottle) float64 4.55 ... ...
    turbidity           (latitude, longitude, time, bottle) float64 0.1969 .....
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 80.89 ......
    scan                (latitude, longitude, time, bottle) float64 5.858e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [65]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [69]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [71]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/17)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 7.13 ... ...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 97.65 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 310.6 ......
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 6.988 ......
    oxy_percentsat_ch2  (latitude, longitude, time, bottle) float64 95.7 ... ...
    oxy_conc_ch2        (latitude, longitude, time, bottle) float64 304.4 ......
    ...                  ...
    temperature_ch1     (latitude, longitude, time, bottle) float64 4.551 ......
    temperature_ch2     (latitude, longitude, time, bottle) float64 4.55 ... ...
    turbidity           (latitude, longitude, time, bottle) float64 0.1969 .....
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 80.89 ......
    scan                (latitude, longitude, time, bottle) float64 5.858e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...

In [73]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/17)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 7.13 ... ...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 97.65 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 310.6 ......
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 6.988 ......
    oxy_percentsat_ch2  (latitude, longitude, time, bottle) float64 95.7 ... ...
    oxy_conc_ch2        (latitude, longitude, time, bottle) float64 304.4 ......
    ...                  ...
    temperature_ch1     (latitude, longitude, time, bottle) float64 4.551 ......
    temperature_ch2     (latitude, longitude, time, bottle) float64 4.55 ... ...
    turbidity           (latitude, longitude, time, bottle) float64 0.1969 .....
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 80.89 ......
    scan                (latitude, longitude, time, bottle) float64 5.858e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
Attributes:
    CruiseID:       DY2103
    VesselName:     R/V Oscar Dyson
    WaterDepth:     90
    date_created:   2021-07-07T09:49:51Z
    date_modified:  
    QC_indicator:   unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [87]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")
    except:
        print(f'Skipping {cast}')

Skipping ctd017test.btl
Skipping ctd021.btl
Skipping ctd079.btl


In [78]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 9, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/17)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 6.789 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 93.85 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 295.5 ......
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 6.713 ......
    oxy_percentsat_ch2  (latitude, longitude, time, bottle) float64 92.81 ......
    oxy_conc_ch2        (latitude, longitude, time, bottle) float64 292.3 ......
    ...                  ...
    temperature_ch1     (latitude, longitude, time, bottle) float64 4.707 ......
    temperature_ch2     (latitude, longitude, time, bottle) float64 4.708 ......
    turbidity           (latitude, longitude, time, bottle) float64 0.2161 .....
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 151.5 ......
    scan                (latitude, longitude, time, bottle) float64 1.247e+04...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
Attributes:
    CruiseID:       DY2103
    VesselName:     R/V Oscar Dyson
    WaterDepth:     156
    date_created:   2021-07-07T09:56:55Z
    date_modified:  
    QC_indicator:   unqcd

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history